***
# Lab 2 ML7331
> ### By Allen Hoskins, Brittany Lewandowski, Alexy Morris, Fidel Tamares


***

In [78]:
#import packages
import pandas as pd
import numpy as np
import lightgbm as lgb

import seaborn as sns
import sklearn as sk
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,SGDClassifier,LogisticRegressionCV
from sklearn.feature_selection import SelectPercentile,f_regression
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

import time
import warnings

import catboost
#allen Packages
from catboost import CatBoostClassifier

In [45]:
#remove warnings after verifying code
warnings.filterwarnings("ignore")

### ***
# Data Preparation 1
Define and prepare your class variables. <br> 
> <font color=Blue> The team used a data set from Kaggle containing attributes about 2015 airline delays. The meaningfulness of the data is to provide both airlines and customers with information relevant to flight delays. This data is useful as it helps customers make decisions about which airline they fly with, and it helps airlines determine what the cause(s) of flight delays are. From a business perspective, knowing this information is critical as it drives an organization’s bottom line.  

> <font color=Blue> To get better insight into the attributes contained in the data set and their data types, the team ran `df.info(verbose=True,show_counts=True)` in Python. 

Use proper variable representations (int, float, one-hot, etc.).<br>

<font color=Blue> Initially starting with the three data sets they contain the following details: <br>
Flights:
> 1. The data set contains 5,819,079 entries.<br> 
> 2. The data set contains 31 columns.  <br>
> 3. There are 16 columns classified with the data type, float.<br>
> 4. There are 10 columns classified with the data type, int. <br>
> 5. There are 5 columns classified with the data type, object.

Airports:
> 1. The data set contains 322 entries.<br> 
> 2. The data set contains 7 columns.  <br>
> 3. There are 2 columns classified with the data type, float.<br>
> 4. There are 5 columns classified with the data type, object.

Airlines:
> 1. The data set contains 14 entries.<br> 
> 2. The data set contains 2 columns.  <br>
> 3. There are 2 columns classified with the data type, object. </font>

Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. <br>
> <font color=Blue>PCA was used preprocessing step to help scales the numeric predictors before fitting out selected models.

Remove variables that are not needed/useful for the analysis
> <font color=Blue>As we analyzed the initial datasets there were a lot of redudnat variables that represented the same type of information. Data points such as IATA_Code (The International Air Transport Association's) to identify an airport and the actual airports name. We also found some data points that we determined would not be important to classifying flight delays such as Flight_Number, cancellation_reason or tail_number
***

In [46]:
df_flights = pd.read_csv(r'/Users/fidel/MSDS7331/data/flightdelay/flights.csv',low_memory = False)

In [47]:
df_flights.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

In [48]:
df_airports = pd.read_csv(r'/Users/fidel/MSDS7331/data/flightdelay/airports.csv',low_memory = False)

In [49]:
df_airports.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   IATA_CODE  322 non-null    object 
 1   AIRPORT    322 non-null    object 
 2   CITY       322 non-null    object 
 3   STATE      322 non-null    object 
 4   COUNTRY    322 non-null    object 
 5   LATITUDE   319 non-null    float64
 6   LONGITUDE  319 non-null    float64
dtypes: float64(2), object(5)
memory usage: 17.7+ KB


In [50]:
df_airlines = pd.read_csv(r'/Users/fidel/MSDS7331/data/flightdelay/airlines.csv',low_memory = False)

In [51]:
df_airlines.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   IATA_CODE  14 non-null     object
 1   AIRLINE    14 non-null     object
dtypes: object(2)
memory usage: 352.0+ bytes


In [52]:
pd.set_option('display.max_columns', None)

In [53]:
#merge flight and airline information
df_merge_airline = df_flights.merge(df_airlines, left_on = 'AIRLINE', right_on = 'IATA_CODE',how='left')
df_merge_airline.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,IATA_CODE,AIRLINE_y
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.


In [54]:
#find airports with missing LATITUDE
df_airports[df_airports['LATITUDE'].isna() == True]

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
96,ECP,Northwest Florida Beaches International Airport,Panama City,FL,USA,NaN,NaN
234,PBG,Plattsburgh International Airport,Plattsburgh,NY,USA,NaN,NaN
313,UST,Northeast Florida Regional Airport (St. August...,St. Augustine,FL,USA,NaN,NaN


In [55]:
#merge origin airport information to merged data 
df_merge_origin = df_merge_airline.merge(df_airports, left_on = 'ORIGIN_AIRPORT',right_on = 'IATA_CODE', how='left')
df_merge_origin.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,IATA_CODE_x,AIRLINE_y,IATA_CODE_y,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.,SFO,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931


In [56]:
rename_dict_origin =  {'CITY':'CITY_ORIGIN',
                'STATE':'STATE_ORIGIN',
                'COUNTRY':'COUNTRY_ORIGIN',
                'LATITUDE':'LATITUDE_ORIGIN',
                'LONGITUDE':'LONGITUDE_ORIGIN'}
df_merge_origin.rename(columns=rename_dict_origin,inplace = True)

df_merge_origin.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,IATA_CODE_x,AIRLINE_y,IATA_CODE_y,AIRPORT,CITY_ORIGIN,STATE_ORIGIN,COUNTRY_ORIGIN,LATITUDE_ORIGIN,LONGITUDE_ORIGIN
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.,SFO,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931


In [57]:
#merge destination airport information to dataset
df_merge_final = df_merge_origin.merge(df_airports,left_on = 'DESTINATION_AIRPORT',right_on = 'IATA_CODE',how='left')
df_merge_final.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,IATA_CODE_x,AIRLINE_y,IATA_CODE_y,AIRPORT_x,CITY_ORIGIN,STATE_ORIGIN,COUNTRY_ORIGIN,LATITUDE_ORIGIN,LONGITUDE_ORIGIN,IATA_CODE,AIRPORT_y,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,PBI,Palm Beach International Airport,West Palm Beach,FL,USA,26.68316,-80.09559
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.,SFO,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484,CLT,Charlotte Douglas International Airport,Charlotte,NC,USA,35.21401,-80.94313
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619


In [58]:
rename_dict_dest =  {'CITY':'CITY_DESTINATION',
                'STATE':'STATE_DESTINATION',
                'COUNTRY':'COUNTRY_DESTINATION',
                'LATITUDE':'LATITUDE_DESTINATION',
                'LONGITUDE':'LONGITUDE_DESTINATION'}
df_merge_final.rename(columns=rename_dict_dest,inplace = True)

df_merge_final.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,IATA_CODE_x,AIRLINE_y,IATA_CODE_y,AIRPORT_x,CITY_ORIGIN,STATE_ORIGIN,COUNTRY_ORIGIN,LATITUDE_ORIGIN,LONGITUDE_ORIGIN,IATA_CODE,AIRPORT_y,CITY_DESTINATION,STATE_DESTINATION,COUNTRY_DESTINATION,LATITUDE_DESTINATION,LONGITUDE_DESTINATION
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,PBI,Palm Beach International Airport,West Palm Beach,FL,USA,26.68316,-80.09559
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.,SFO,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484,CLT,Charlotte Douglas International Airport,Charlotte,NC,USA,35.21401,-80.94313
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619


In [59]:
df_merge_final.columns= df_merge_final.columns.str.strip().str.lower()

In [60]:
df_ordered = df_merge_final.copy(deep=True)

In [61]:
col_order = ['year','month','day','day_of_week','airline_x','airline_y','flight_number','tail_number',
             'origin_airport','airport_x','city_origin','state_origin','country_origin','latitude_origin','longitude_origin',
             'destination_airport','airport_y','city_destination','state_destination','country_destination','latitude_destination','longitude_destination',
             'scheduled_departure','departure_time','departure_delay','taxi_out','wheels_off','scheduled_time','elapsed_time','air_time','distance',
             'wheels_on','taxi_in','scheduled_arrival','arrival_time','arrival_delay','diverted','cancelled','cancellation_reason','air_system_delay',
             'security_delay','airline_delay','late_aircraft_delay','weather_delay']

In [62]:
df_ordered = df_ordered[col_order]

In [63]:
rename_dict_merged =  {'airline_x':'airline_cd',
                       'airline_y':'airline_name',
                       'airport_x': 'airport_origin',
                       'airport_y':'airport_destination'}
df_ordered.rename(columns=rename_dict_merged,inplace = True)

In [64]:
df_ordered.to_csv(r'/Users/fidel/MSDS7331/data/flightdelay/airlines_merged.csv')

***
# Data Preparation 2
Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).

<font color=Blue> The final data set that we started our analysis with was named airlines_merged and contained the following details:
> 1. The data set contains 5,819,079 entries.<br> 
> 2. The data set contains 45 columns.  <br>
> 3. There are 20 columns classified with the data type, float.<br>
> 4. There are 11 columns classified with the data type, int. <br>
> 5. There are 14 columns classified with the data type, object.

The __5,819,079x45 Data Frame__ is sufficient size for us to conduct our analysis and classification of future flights. <br>
We have also included a data ditionary to better explain the purpose of each predictor. 
***

In [65]:
df = pd.read_csv(r'/Users/fidel/MSDS7331/data/flightdelay/airlines_merged.csv',low_memory=False)

In [66]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 45 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Unnamed: 0             int64  
 1   year                   int64  
 2   month                  int64  
 3   day                    int64  
 4   day_of_week            int64  
 5   airline_cd             object 
 6   airline_name           object 
 7   flight_number          int64  
 8   tail_number            object 
 9   origin_airport         object 
 10  airport_origin         object 
 11  city_origin            object 
 12  state_origin           object 
 13  country_origin         object 
 14  latitude_origin        float64
 15  longitude_origin       float64
 16  destination_airport    object 
 17  airport_destination    object 
 18  city_destination       object 
 19  state_destination      object 
 20  country_destination    object 
 21  latitude_destination   float64
 22  longitude_destinat

| Attribute Name | Attribute Description | Non-null Count | Attribute Data Type|
| :--- | :--- | :---| :---|
| year | Year data was collected |5819079|int |
| month | Month data was collected |5819079| int |
| day | Day data was collected (1-25) |5819079| int |
| day_of_week | Day of the week data was collected (1-7) |5819079|int |
| airline_name | Name of airline ex) Alaska Airlines Inc. |5819079| object |
| airline_cd | Code for airline ex) AA |5819079|object |
| flight_number | Unique identifier for each flight |5804358| int |
| tail_number  | Unique identifier for a plane | 5819079|object |
| origin_airport | Code of origin airport ex) DFW for Dallas Fort Worth International Airport |5819079| object |
| airport_origin | Airport flight originates from ex) San Francisco International Airport  | 5332914|object  |
| city_origin | City the flight came from | 5332914|object |
| state_origin | State the flight came from | 5332914|object |
| country_origin | Country flight came from | 5332914|object |
| latitude_origin | Latitude coordinates the flight came from | 5332914|float |
| longitude_origin | Longitude coordinates the flight came from | 5332914|float |
| destination_airport | Code of destination airport ex) DFW for Dallas Fort Worth International Airport |5819079| object |
| airport_destination | Name of destination airport ex) Dallas Fort Worth International Airport |5332914|	object |
| city_destination | City of destination | 5332914|object |
| state_destination | State of destination | 5332914|object |
| country_destination | Country of destination |5332914| object |
| latitude_destination | Latitude coordinates of flight destination | 5332914|float |
| longitude_destination | Longitude coordinates of flight destination |5332914| float |
| scheduled_departure | Time of flight departure | 5819079|int |
| departure_time | Time flight departed | 5732926|float |
| departure_delay | Time flights are delayed from departure |5732926| float |
| taxi_out | Time spent taxiing to the runway  |5730032| float |
| wheels_off | Time spent removing wheels  | 5730032|float  |
| scheduled_time | When flight is scheduled to back up from gate | 5819073|float |
| elapsed_time | Calculated column derived by adding taxi & air time together |5714008| float |
| air_time | Time flight spends in the air | 5714008|float |
| distance | Distance traveled by flights | 5819079|int |
| wheels_on | Time spent |5726566| float |
| taxi_in | Time spent taxiing to the gate | 5726566|float |
| scheduled_arrival | Scheduled flight arrival time |5819079| int |
| arrival_time | Time flight actually arrived | 5726566|float |
| arrival_delay |Time flights are delayed from arrival |5714008| float |
| diverted | Binary indicator of if a flight was diverted or not (0,1) |5819079| int |
| cancelled | Binary indicator of if a flight was canceled or not (0,1) |5819079| int |
| cancellation_reason | Reason for flight cancellation | 89884|object |
| air_system_delay | Flight delays due to air systems |1063439| float |
| security_delay | Flight delays due to security issues |1063439| float |
| airline_delay | Flight delays due to an airline specific problem |1063439| float |
| late_aircraft_delay | Flight delays due to an aircraft being late | 5819079|float |
| weather_delay  | Flight delays due to weather | 1063439|float |

***

In [67]:
pd.set_option('display.max_columns', None)

In [68]:
#removing unwanted column from previous export
df = df.drop(columns =['Unnamed: 0'],axis =1)

In [69]:
#dropping highly correlated columns
#https://www.codegrepper.com/code-examples/python/how+to+drop+highly+correlated+features
# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))

# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

df = df.drop(columns = to_drop, axis=1)

#looking at dataframe after removing highly correlated fields.
df.info()

# Modeling and Evaluations 1
Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). <br>

> <font color=Blue>Our 4 evaluation metrics generated by the confusion matrix we decided to make recall our primary evaluation metric while still taking accuracy, precision and F1 into consideration </font>

Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation 
backing up any assertions. <br>

> <font color=Blue> We decided to use recall as our primary because we determined that False Negatives (FN) have a greater impact on a travel than False Positivers (FP). If a travel experiences an FP they may be frustrated but their travel schedule will not be impacted but in the case of a FN the travel is not expecting a delay but will encounter one at the last momemnt.

# Modeling and Evaluations 2 
Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?) <br>
> <font color=Blue> The method we decided to use to create our training and testing data sets was ____
    
Explain why your chosen method is appropriate or use more than one method as appropriate.

# Modeling and Evaluations 3 
Create three different classification/regression models (e.g., random forest, KNN, and SVM).
> <font color=Blue> We have selected a logistic regression model to set our baseline for evaluation, random forests and KNN
    
Two modeling techniques must be new (but the third could be SVM or logistic regression).
Adjust PARAMETERS as appropriate to increase generalization performance using your chosen metric.
> <font color=Blue> The perameters we adjusted and why in each model were: <br>
> Logistic Regression: <br>
> KNN: <br>
> Random Forests: <br>

In [70]:
#selecting subset of data set
airports = ['BHM']

In [71]:
df_air = df.loc[df['origin_airport'].isin(airports)]

In [72]:
df_air.describe()

,year,month,day,day_of_week,flight_number,latitude_origin,longitude_origin,latitude_destination,longitude_destination,scheduled_departure,departure_delay,taxi_out,scheduled_time,wheels_on,taxi_in,scheduled_arrival,diverted,cancelled,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
count,12514.0,12514.000000,12514.000000,12514.000000,12514.000000,1.251400e+04,1.251400e+04,12514.000000,12514.000000,12514.000000,12338.000000,12334.000000,12514.000000,12324.000000,12324.000000,12514.000000,12514.000000,12514.000000,2128.000000,2128.000000,2128.000000,2128.000000,2128.000000
mean,2015.0,6.150951,15.687710,3.879575,2876.476346,3.356294e+01,-8.675355e+01,34.252718,-88.175001,1192.116909,9.032906,13.783525,108.121864,1399.118306,8.632587,1416.011507,0.004075,0.014464,14.805921,0.091635,20.292293,27.732613,2.580827
std,0.0,3.384375,8.763785,1.992624,1951.952017,7.794965e-12,1.647104e-11,4.990106,8.325266,445.126945,43.869223,7.389709,36.974468,458.707388,5.844535,443.946518,0.063711,0.119397,32.820428,3.167858,73.036267,47.048389,16.190917
min,2015.0,1.000000,1.000000,1.000000,3.000000,3.356294e+01,-8.675355e+01,25.793250,-115.152330,500.000000,-20.000000,4.000000,52.000000,3.000000,1.000000,707.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2015.0,3.000000,8.000000,2.000000,1051.000000,3.356294e+01,-8.675355e+01,29.980470,-95.339720,745.000000,-5.000000,9.000000,76.000000,952.000000,5.000000,1009.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2015.0,6.000000,16.000000,4.000000,2525.000000,3.356294e+01,-8.675355e+01,33.640440,-84.426940,1200.000000,-2.000000,12.000000,113.000000,1402.000000,7.000000,1414.000000,0.000000,0.000000,3.000000,0.000000,0.000000,9.000000,0.000000
75%,2015.0,9.000000,23.000000,5.000000,4592.000000,3.356294e+01,-8.675355e+01,39.175400,-83.348840,1609.000000,4.000000,16.000000,123.000000,1823.000000,10.000000,1825.000000,0.000000,0.000000,18.000000,0.000000,10.000000,34.000000,0.000000
max,2015.0,12.000000,31.000000,7.000000,6813.000000,3.356294e+01,-8.675355e+01,42.212060,-73.872610,2055.000000,1988.000000,116.000000,260.000000,2357.000000,150.000000,2325.000000,1.000000,1.000000,493.000000,144.000000,1971.000000,420.000000,240.000000


In [73]:
#selecting columns that are dtype != numeric
non_numeric = df_air.select_dtypes(exclude = np.number).columns

>Because we want to predict if an airplane is going to be delayed or not using logistic regression, we created a new field called `is_delay` from the `departure_delay` field. To do this, we created a function that we could pass a numeric field through and if the value for that row is greater than 0, we imputed "1" into the curated field to signifiy the flight was delayed. If the value was less than 0, we imputed "0" to signify that the flight was not delayed. 

In [86]:
#create binary response for logistic regression
# create a function
def is_delayed(delay,axis=1):
    if delay <= 0:
        return 0
    else:
        return 1
    
# create a new column based on condition
df_air['is_departure_delay'] = df_air.loc[:,'departure_delay'].apply(is_delayed)

After creating the is_delay field, we then began to One Hot Encode any categorial variables so that our model could interpret and use those columns. This caused a 250k by 42 data set to become 240k by 3400.

In [75]:
df_air.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12514 entries, 321 to 5817010
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   12514 non-null  int64  
 1   month                  12514 non-null  int64  
 2   day                    12514 non-null  int64  
 3   day_of_week            12514 non-null  int64  
 4   airline_cd             12514 non-null  object 
 5   airline_name           12514 non-null  object 
 6   flight_number          12514 non-null  int64  
 7   tail_number            12510 non-null  object 
 8   origin_airport         12514 non-null  object 
 9   airport_origin         12514 non-null  object 
 10  city_origin            12514 non-null  object 
 11  state_origin           12514 non-null  object 
 12  country_origin         12514 non-null  object 
 13  latitude_origin        12514 non-null  float64
 14  longitude_origin       12514 non-null  float64
 15

In [88]:
df_air.isna().sum().sum()

65003

In [87]:
#Departure Delay - response
#altered from https://www.statology.org/one-hot-encoding-in-python/

#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='error',drop='first')

#perform one-hot encoding on non-numeric column s
encoder_df = pd.DataFrame(encoder.fit_transform(df_air[non_numeric]).toarray())

#return column names from encoder
encoder_df.columns = encoder.get_feature_names_out()

#merge one-hot encoded columns back with original DataFrame
encoded = df_air.join(encoder_df)

ValueError: Input contains NaN

In [ ]:
#drop original columns
final_df = encoded.drop(non_numeric, axis=1)

In [ ]:
#remove target variable and column target was created from training data set
target_delay_cols = ['is_departure_delay','departure_delay']
feature_delay_cols = final_df.loc[:, ~final_df.columns.isin(target_delay_cols)].columns
X_delay = final_df[feature_delay_cols] # Features
y_delay = final_df.is_delay # Target variable

In [ ]:
#remove target variable and column target was created from training data set
target_cancel_cols = ['cancelled','cancellation_reason']
feature_cancel_cols = final_df.loc[:, ~final_df.columns.isin(target_cancel_cols)].columns
X_cancel = final_df[feature_cancel_cols] # Features
y_cancel = final_df.is_delay # Target variable

In [ ]:
#split original dataset into features and target variable
X_delay_train,X_delay_test,y_delay_train,y_delay_test=train_test_split(X_delay,y_delay,test_size=0.20,random_state=0)

In [ ]:
#split original dataset into features and target variable
X_cancel_train,X_cancel_test,y_cancel_train,y_cancel_test=train_test_split(X_cancel,y_cancel,test_size=0.20,random_state=0)

In [81]:
#imputing 0 into NaN values for logistic regression after OHE
X_cancel_train = X_cancel_train.fillna(0)
X_cancel_test = X_cancel_test.fillna(0)
y_cancel_train = y_cancel_train.fillna(0)
y_cancel_test = y_cancel_test.fillna(0)

NameError: name 'X_cancel_train' is not defined

In [ ]:
#imputing 0 into NaN values for logistic regression after OHE
X_delay_train = X_delay_train.fillna(0)
X_delay_test = X_delay_test.fillna(0)
y_delay_train = y_delay_train.fillna(0)
y_delay_test = y_delay_test.fillna(0)

In [ ]:
#checking shape of data
print('X_cancel_train shape: ',X_cancel_train.shape,'\n')
print('X_cancel_test shape: ',X_cancel_test.shape,'\n')
print('y_cancel_train shape: ',y_cancel_train.shape,'\n')
print('y_cancel_test shape: ',y_cancel_test.shape)

In [ ]:
#checking shape of data
print('X_delay_train shape: ',X_delay_train.shape,'\n')
print('X_delay_test shape: ',X_delay_test.shape,'\n')
print('y_delay_train shape: ',y_delay_train.shape,'\n')
print('y_delay_test shape: ',y_delay_test.shape)

### Logistic Regression:
To simplify the code and useability of the model, we used a pipeline function from sklearn to standardize (using StandarScaler), run the data through PCA and then finally a logistic regession model using the PCA output.
***

In [80]:
#https://towardsdatascience.com/getting-the-most-out-of-scikit-learn-pipelines-c2afc4410f1a

# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
pca = PCA()
# Define a Standard Scaler to normalize inputs
scaler = StandardScaler()

tolerance = 0.1
l1ratio = 1
rs = 0

# Scale -> PCA -> LR 
logistic = LogisticRegression(solver = 'saga',
                                random_state = rs,
                                penalty ='elasticnet',
                                l1_ratio = l1ratio,
                                tol = tolerance)

pipe = Pipeline(steps=[("scaler", scaler),
                       ("pca", pca),
                       ("logistic", logistic)]
               )

pipe.fit(X_cancel_train,y_cancel_train)

NameError: name 'X_cancel_train' is not defined

In [ ]:
#predict
y_pred=pipe.predict(X_test)

After creating predictions on our test data set, we pulled metrics to determine how well our model performed. Our base model had an accuracy of 78.45% with a precision of 95.09% and a recall of 49.02%. This model output gives us a good starting point for hypertuing the model and comparison against some more powerful models such as XGBoost or LightGBM. After computing these metrics we then created a confusion matrix heatmap to better visualize how well our model was predicting and where the model could perform better with False Positives and False Negatives.

In [42]:
#Metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

NameError: name 'y_test' is not defined

### LGBM:
***

In [19]:
#https://towardsdatascience.com/getting-the-most-out-of-scikit-learn-pipelines-c2afc4410f1a
#https://lightgbm.readthedocs.io/en/latest/Parameters.html#objective

# Define a pipeline to search for the best combination of PCA truncation

pca_lgbm = PCA(n_components = .95,
               svd_solver = 'auto')


In [20]:
# Define a Standard Scaler to normalize inputs

scaler_lgbm = RobustScaler()

lr = 0.15
boost = 'dart' #'gbdt' 
obj = 'binary'
tl = 'voting' # 'data'
numiter = 1000
mbin = 300
nleaves = 42


In [21]:
# Scale -> PCA -> LR 
lgbm = lgb.LGBMClassifier(learning_rate = lr,
                          boosting = boost,
                          objective = obj,
                          tree_learner = tl,
                          #num_iterations = numiter,
                          num_leaves = nleaves,
                          max_bin = mbin,
                          extra_trees = True)


In [22]:
%%time
#start_lgbm = time.time()
pipe_lgbm = Pipeline(steps=[("scaler", scaler_lgbm),
                       ("pca", pca_lgbm),
                       ("LightGBM", lgbm)])

pipe_lgbm.fit(X_train,y_train)
#end_lgbm = time.time()

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
CPU times: user 23.8 s, sys: 3.74 s, total: 27.6 s
Wall time: 4.6 s


Pipeline(steps=[('scaler', RobustScaler()), ('pca', PCA(n_components=0.95)),
                ('LightGBM',
                 LGBMClassifier(boosting='dart', extra_trees=True,
                                learning_rate=0.15, max_bin=300, num_leaves=42,
                                objective='binary', tree_learner='voting'))])

In [23]:
#predict
y_pred_lgbm=pipe_lgbm.predict(X_test)

In [24]:
accuracy_lgbm = metrics.accuracy_score(y_test,y_pred_lgbm)
precision_lgbm = metrics.precision_score(y_test, y_pred_lgbm)
recall_lgbm = metrics.recall_score(y_test, y_pred_lgbm)

In [25]:
#print results 
print('____________Start LightGBM Base Model Results______________:')
print('Accuracy:\n',accuracy_lgbm,'\n')
print('Precision:\n',recall_lgbm,'\n')
print('Recall:\n',recall_lgbm,'\n')
print('____________End LightGBM Base Model Results________________')

____________Start LightGBM Base Model Results______________:
Accuracy:
 0.8286056731921694 

Precision:
 0.4732824427480916 

Recall:
 0.4732824427480916 

____________End LightGBM Base Model Results________________


***
## Random Forest:
***

In [26]:
#Importing required libraries.
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

In [27]:
#Specififying Stratified Kfold for cv.
skfcv = StratifiedKFold(n_splits=10)

In [28]:
#Creating our random forest pipeline.
rf = RandomForestClassifier(n_estimators=100, random_state=0)

In [29]:
#Fitting random forest model on training data.
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [30]:
#Predicting on our test set.
y_pred_rf = rf.predict(X_test)

In [31]:
#Code adapted from: https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652
rfc_cv_score = cross_val_score(rf, X_test, y_test, cv=skfcv, scoring='accuracy')


In [32]:
#Code adapted from: https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred_rf))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, y_pred_rf))
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[1660   57]
 [ 330  456]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.83      0.97      0.90      1717
           1       0.89      0.58      0.70       786

    accuracy                           0.85      2503
   macro avg       0.86      0.77      0.80      2503
weighted avg       0.85      0.85      0.83      2503

=== All AUC Scores ===
[0.812749   0.83266932 0.84063745 0.832      0.856      0.852
 0.808      0.84       0.8        0.832     ]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.830605577689243


# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.model_selection import StratifiedKFold

In [ ]:
scaler_knn = RobustScaler()

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)

pipe_knn = Pipeline(steps=[("scaler", scaler_knn),
                       ("KNN", knn)])

pipe_knn.fit(X_train,y_train)

y_pred_pipe_knn = pipe_knn.predict(X_test)

pipe_knn_cv_score = cross_val_score(pipe_knn, X_test, y_test, cv=skfcv, scoring='accuracy')

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred_pipe_knn))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, y_pred_pipe_knn))
print("=== All AUC Scores ===")
print(pipe_knn_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - KNN: ", pipe_knn_cv_score.mean())

In [ ]:
knn2 = KNeighborsClassifier(n_neighbors = 3, weights='distance')

pipe_knn2 = Pipeline(steps=[("scaler", scaler_knn),
                       ("KNN", knn2)])

pipe_knn2.fit(X_train,y_train)

y_pred_pipe_knn2 = pipe_knn2.predict(X_test)

pipe_knn2_cv_score = cross_val_score(pipe_knn2, X_test, y_test, cv=skfcv, scoring='accuracy')

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred_pipe_knn2))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, y_pred_pipe_knn2))
print("=== All AUC Scores ===")
print(pipe_knn2_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - KNN: ", pipe_knn2_cv_score.mean())

In [ ]:
knn3 = KNeighborsClassifier(n_neighbors = 5, weights='distance')

pipe_knn3 = Pipeline(steps=[("scaler", scaler_knn),
                       ("KNN", knn3)])

pipe_knn3.fit(X_train,y_train)

y_pred_pipe_knn3 = pipe_knn3.predict(X_test)

pipe_knn3_cv_score = cross_val_score(pipe_knn3, X_test, y_test, cv=skfcv, scoring='accuracy')

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred_pipe_knn3))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, y_pred_pipe_knn3))
print("=== All AUC Scores ===")
print(pipe_knn3_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - KNN: ", pipe_knn3_cv_score.mean())

In [ ]:
knn4 = KNeighborsClassifier(n_neighbors = 3, weights='distance', algorithm='kd_tree', leaf_size=60)

pipe_knn4 = Pipeline(steps=[("scaler", scaler_knn),
                       ("KNN", knn4)])

pipe_knn4.fit(X_train,y_train)

y_pred_pipe_knn4 = pipe_knn4.predict(X_test)

pipe_knn4_cv_score = cross_val_score(pipe_knn4, X_test, y_test, cv=skfcv, scoring='accuracy')

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred_pipe_knn4))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, y_pred_pipe_knn4))
print("=== All AUC Scores ===")
print(pipe_knn4_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - KNN: ", pipe_knn4_cv_score.mean())

# Gridsearch

In [45]:
#Specifying the number of trees in random forest:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 10)]
#Specifying criterion for each split:
criterion = ['gini','entropy','log_loss']
#Specifying the maximum number of levels in the tree:
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
#Specifying the number of features to consider at every split:
max_features = ['auto','sqrt', 'log2','none']
#Specifying the minimum number of samples required to split a node:
min_samples_split = [2, 5, 10]
#Specifying the minimum number of samples required at each leaf node:
min_samples_leaf = [1, 2, 4]
#Specifying the method of selecting samples for training each tree:
bootstrap = [True, False]
#Creating and printing my random grid with parameters defined above:

njobs = [-1]
verbo = [1]
param_grid= {'n_estimators': n_estimators,
               'criterion': criterion,
               'verbose': verbo,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'n_jobs': njobs
            }

print(param_grid)

#Base RF model used to tune:
rf = RandomForestClassifier(random_state=0)
rf_random = GridSearchCV(estimator = rf, param_grid = param_grid, cv = skfcv)
#Fiting the random search model on our training data:
rf_random.fit(X_train, y_train)
print(rf_random.best_estimator_)

{'n_estimators': [100, 144, 188, 233, 277, 322, 366, 411, 455, 500], 'criterion': ['gini', 'entropy', 'log_loss'], 'verbose': [1], 'max_features': ['auto', 'sqrt', 'log2', 'none'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False], 'n_jobs': [-1]}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.9s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.3s
[Parallel(n

KeyboardInterrupt: 

In [38]:
print(rf_random.best_estimator_)

RandomForestClassifier(bootstrap=False, max_depth=50, min_samples_split=5,
                       random_state=0)


# CatBoost

In [ ]:
df_air_test = df_air.copy()

In [46]:
#convert object fields and category fields 
categories = ['year','month','day','day_of_week','airline_cd','airline_name','flight_number','origin_airport','tail_number',
              'airport_origin','city_origin','state_origin','country_origin','destination_airport','airport_destination',
              'city_destination','state_destination','country_destination','cancellation_reason']

df_air[categories] = df_air[categories].fillna('Unknown')

/var/folders/cn/_lgzxbk177j14_b82nbl10br0000gn/T/ipykernel_87755/493185035.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_air[categories] = df_air[categories].fillna('Unknown')


In [47]:
df_air[categories] = df_air[categories].astype('category')

/var/folders/cn/_lgzxbk177j14_b82nbl10br0000gn/T/ipykernel_87755/838760448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_air[categories] = df_air[categories].astype('category')


In [48]:
df_air.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12514 entries, 321 to 5817010
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   year                   12514 non-null  category
 1   month                  12514 non-null  category
 2   day                    12514 non-null  category
 3   day_of_week            12514 non-null  category
 4   airline_cd             12514 non-null  category
 5   airline_name           12514 non-null  category
 6   flight_number          12514 non-null  category
 7   tail_number            12514 non-null  category
 8   origin_airport         12514 non-null  category
 9   airport_origin         12514 non-null  category
 10  city_origin            12514 non-null  category
 11  state_origin           12514 non-null  category
 12  country_origin         12514 non-null  category
 13  latitude_origin        12514 non-null  float64 
 14  longitude_origin       12514 non-n

In [ ]:
df_air['tail_number'] = df_air['tail_number'].astype('category')
df_air['airport_origin'] = df_air['airport_origin'].astype('category')

In [49]:
#remove target variable and column target was created from training data set
target_cols = ['is_delay','departure_delay']
feature_cols = df_air.loc[:, ~df_air.columns.isin(target_cols)].columns
Xcat = df_air[feature_cols] # Features
ycat = df_air.is_delay # Target variable

In [50]:
#split original dataset into features and target variable
Xcat_train,Xcat_test,ycat_train,ycat_test=train_test_split(Xcat,ycat,test_size=0.20,random_state=0)

In [51]:
#checking shape of data
print('X_train shape: ',Xcat_train.shape,'\n')
print('X_test shape: ',Xcat_test.shape,'\n')
print('y_train shape: ',ycat_train.shape,'\n')
print('y_test shape: ',ycat_test.shape)

X_train shape:  (10011, 36) 

X_test shape:  (2503, 36) 

y_train shape:  (10011,) 

y_test shape:  (2503,)


In [52]:
cat_feat = df_air.select_dtypes(include=['category']).columns.tolist()

In [56]:
%%time
#https://www.kaggle.com/code/mitribunskiy/tutorial-catboost-overview/notebook



seed = 0
params = {
    #'loss_function':'Logloss', # objective function
          'eval_metric':'Accuracy', # metrics
          'verbose': 200, # output to stdout info about training process every 200 iterations
          'random_seed': seed,
          'learning_rate': .25,
          'cat_features': cat_feat
         }

cbc = CatBoostClassifier(**params)
cbc.fit(Xcat_train, ycat_train, # data to train on (required parameters, unless we provide X as a pool object, will be shown below)
          eval_set=(Xcat_test, ycat_test), # data to validate on
          use_best_model=True, # True if we don't want to save trees created after iteration with the best validation score
          plot=True # True for visualization of the training process (it is not shown in a published kernel - try executing this code)
         );

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.8121067	test: 0.8202157	best: 0.8202157 (0)	total: 18.3ms	remaining: 18.2s
200:	learn: 0.9070023	test: 0.8541750	best: 0.8581702 (178)	total: 4.8s	remaining: 19.1s
400:	learn: 0.9538508	test: 0.8573712	best: 0.8581702 (178)	total: 8.96s	remaining: 13.4s
600:	learn: 0.9785236	test: 0.8585697	best: 0.8605673 (450)	total: 12.9s	remaining: 8.55s
800:	learn: 0.9910099	test: 0.8613664	best: 0.8629644 (738)	total: 16.7s	remaining: 4.16s
999:	learn: 0.9974029	test: 0.8565721	best: 0.8637635 (885)	total: 20.6s	remaining: 0us

bestTest = 0.8637634838
bestIteration = 885

Shrink model to first 886 iterations.
CPU times: user 43.5 s, sys: 7.74 s, total: 51.2 s
Wall time: 20.9 s


# Modeling and Evaluations 4 
Analyze the results using your chosen method of evaluation.
> To summarize the performance our models:


Use visualizations of the results to bolster the analysis.
Explain any visuals and analyze why they are interesting to someone that might use this model

# Modeling and Evaluations 5 (place holder)
Discuss the advantages of each model for each classification task, if any. <br>
If there are not advantages, explain why. <br>
Is any model better than another? <br>
Is the difference significant with 95% confidence? MUST Use proper statistical comparison methods

# Modeling and Evaluations 6 (place holder)
Which attributes from your analysis are most important? <br> 
Use proper methods discussed in class to evaluate the importance of different attributes. <br> 
Discuss the results and hypothesize about why certain attributes are more important than others 
for a given classification task (This also includes regression)

***
# Deployment
"How useful is your model for interested parties (i.e., the companies or 
organizations that might want to use it for prediction)? <br>
 > <font color= blue> Our model would be useful for travelers and airlines. For travellers, having a flight delayed or cancelled can impact their travel plans, especially if there are connecting flights. Being able to identify airlines with higher rates of delays and cancellations can help them make better decisions. As an airline, this could be useful in identifying the major indicators in the different types of delays. For every airline flight cancelled or delayed is money that cannot be recouped and can result in additional paid compensation (lost luggage/ lodging expenses/ food vouchers) while still having to resolve the issue with the customer. <br>
 
 
How would you measure the model's value if it was used by these parties? How would your deploy your model for 
interested parties? <br>
> <font color= blue> We would measure the models value as a cost savings to both travelers and airlines. This would be a simple model to deploy and since it uses flight records. Maintaining a database that appends new records could provide timely data for both parties in avoiding travel disruptions and providing flags that may add additional costs to each flight. <br>


What other data should be collected?<br>
> <font color= blue> Other datapoints that could be useful to add value to the model would be: <br>


| # | Feature Addition | Description | Use Case |
| :--- | :--- | :--- |:--- |
| 1 | Aircraft Model | Plane Models  | This could be used to better understand flight efficiency based on age of and plane sophistication |
| 2 | Maintenance History | Last maintenance check, frequency of tune ups, number of major overhauls | Maintenance history could help determine how fast a plane gets in the air, the speeds it can reach and the turn around from one flight to another |
| 3 | Customer Satisfaction Score | We imagine a survey score on different categories like comfort, entertainment, etc | Customer satisfaction scores could be used to target high priority issues the resolve. An example would be If customers rated the flight with a low timeliness score, turn-around procedures would be identified and reviewed |
| 4 | Pilot Years of Experience | Measured in years or months,  could be categorized by position title  | Our assumption is pilot years of experience may influence how long it takes to get planes off the ground and how they handle weather or flight routes |
| 5 | Flight Costs | Measured in Dollars | This could be a measure for airlines that have little to no delay. Our assumption is punctuality and timeliness could influence consumer trust in an airline, the higher the demand would then in turn result in a higher price. Finding that influences flight costs may be correlated with flight efficiency |
| 6 | Flight Weight | Numeric: in lbs or kg  | Our assumption is a heavier plane takes longer to load and could influence the flight speed which would influence departure and arrival delays  |


How often would the model need to be updated, etc.? <br>
> The model would not need major updates unless that is a signiciant change to the industry through regulation or technoloical changes. The model is based on past records that share similar economic and political environments as well as analyzing seasonal travel behavior. If these environments do not change the model should be resilient as time goes on.
***

***
# Exceptional Work
• You have free reign to provide additional modeling. 
• One idea: grid search parameters in a parallelized fashion and visualize the 
performances across attributes. Which parameters are most significant for making a 
good model for each classification algorithm
***